In [1]:
import math
import numpy as np
import os
import pandas as pd
import pysurvival
import rpy2.robjects as robjects
import rpy2.robjects.packages as rpackages
from statistics import mean

from pysurvival.utils.display import correlation_matrix
from pysurvival.models.survival_forest import RandomSurvivalForestModel
from pysurvival.utils.metrics import concordance_index, integrated_brier_score

from sklearn.model_selection import StratifiedKFold

from statsmodels.stats.outliers_influence import variance_inflation_factor

from lifelines import CoxPHFitter

In [ ]:
# Only need to run this once
# utils = rpackages.importr('utils')
# utils.chooseCRANmirror(ind=1)
# utils.install_packages("survAUC")

### Feature Selection

In [13]:
def cox_feature_select(X, t, e):
    num_samples = X.shape[0]
    num_events = sum(e)

    data_table = X
    data_table['Time'] = t
    data_table['Event'] = e

    max_num_features = math.ceil(num_events/10)
    cph = CoxPHFitter()
    cph.fit(data_table, duration_col='Time', event_col='Event')

    hr = abs(cph.params_)
    
    filtered_hr = hr.nlargest(n=max_num_features, keep='first')
    index_names = filtered_hr.index

    col_names = []
    for x in range(1, len(index_names)):
        col_names.append(index_names[x])

    filtered_X = X.filter(items=col_names, axis=1)
    print("Cox filter remaining variables: \n", filtered_X.columns)

    return filtered_X

In [3]:
def calculate_vif(X, thresh=10):
    variables = list(range(X.shape[1]))
    dropped=True

    while dropped:
        dropped=False
        vif = [variance_inflation_factor(X.iloc[:, variables].values, ix) \
               for ix in range(X.iloc[:, variables].shape[1])]
    
        maxloc = vif.index(max(vif))
        if max(vif) > thresh:
            # print("Dropping \'" + X.iloc[:, variables].columns[maxloc] + "\' at index: "+ str(maxloc))
            del variables[maxloc]
            dropped = True
    
    print('VIF remaining variables:')
    print(X.columns[variables])
    return X.iloc[:, variables]
        


### Model Creation and Training

In [4]:
def gh_c_index(risk_pred):
    """
    Calculate Gonen and Hiller's c-index using function from R (using rpy2) 

    Args:
        risk_pred: np.ndarray or torch.Tensor, risk score predictions from model

    Source: Gonen, M. and G. Heller (2005). 
    Concordance probability and discriminatory power in proportional hazards regression.
    Biometrika 92, 965–970.
    """

    # check for NaNs
    if not isinstance(risk_pred, np.ndarray):
        risk_pred = risk_pred.detach().cpu().numpy()
    for a in risk_pred:
        if np.isnan(a).any():
            raise ValueError("NaNs detected in inputs, please correct or drop.")

    # Use Gonen and Hiller's c-index via the survAUC library in R
    survAUC = rpackages.importr('survAUC')

    # Get data into right format
    R_risk_pred = robjects.vectors.FloatVector(risk_pred)

    # this doesn't work yet, need to get the list to numeric type
    # in R, this is accomplished with as.numeric and unlist()
    R_cind = survAUC.GHCI(R_risk_pred)

    # Convert back to Python list with single value
    cind = list(R_cind)

    # Return the only value in the cind list
    return cind[0]

In [5]:
def train_survival_model(X, t, e, num_trees, max_depth, min_node_size, seed=16):
    """
    Function to create and run Random Survival Forest with given attributes on data.

    Args:
        X: array -- input features, rows as samples
        t: array -- time labels for X, when event of interest or censoring occurred
        e: array -- event labels for X, if event occurred (1=event, 0=censoring)
        num_trees: int -- number of trees that will be built in forest model, used in initialization of model
        max_depth: int -- maximum number of levels allowed in tree, used in model fit
        min_node_size: int -- minimum number of samples required to be at leaf node, used in model fit
        seed: int -- random seed used by random number generator in model fit

    Returns: 
        rsf: pysurvival.model.RandomSurvivalForestModel -- model fit to input data
    """

    # Create instance of the model
    rsf = RandomSurvivalForestModel(num_trees=num_trees)

    # Fit model to data
    # Arguments not used from function input are defaults except importance_mode
    # TODO: need to find out what importance mode is 
    rsf.fit(X, t, e, max_features='all', max_depth=max_depth, min_node_size=min_node_size,
            num_threads=-1, sample_size_pct=0.63,
            seed=seed, save_memory=False)


    return rsf

In [6]:
def evaluate_forest(rsf, XT, tT, eT):
    
    risk = rsf.predict_risk(XT)
    h_c_ind = concordance_index(rsf, XT, tT, eT)
    gh_c_ind = gh_c_index(risk)
    ibs = integrated_brier_score(rsf, XT, tT, eT)

    return h_c_ind, gh_c_ind, ibs, risk


In [7]:
def kfold_train_survival_model(X, t, e, num_trees, max_depth, min_node_size, k=5, seed=16):
    CI = []
    GHCI = []
    IBS = []
    STD = []
    best_CI = 0
    best_GHCI = 0
    best_IBS = 0 
    best_fold_rsf = None

    kf = StratifiedKFold(n_splits=k, random_state=seed, shuffle=True)

    for fold, (train_idx, val_idx) in enumerate(kf.split(X, e)):
        # Output current fold number
        # print('Fold {}'.format(fold + 1))

        X_train, X_val = X.loc[train_idx], X.loc[val_idx]
        t_train, t_val = t.loc[train_idx], t.loc[val_idx]
        e_train, e_val = e.loc[train_idx], e.loc[val_idx]

        fold_rsf = train_survival_model(X_train, t_train, e_train, num_trees, max_depth, min_node_size, seed)

        fold_h_c_ind, fold_gh_c_ind, fold_ibs, _ = evaluate_forest(fold_rsf, X_val, t_val, e_val)

        CI.append(fold_h_c_ind)
        GHCI.append(fold_gh_c_ind)
        IBS.append(fold_ibs)

        if best_CI < fold_h_c_ind:
            best_CI = fold_h_c_ind
            best_GHCI = fold_gh_c_ind
            best_IBS = fold_ibs
            best_fold_rsf = fold_rsf

    CI_avg = mean(CI)
    GHCI_avg = mean(GHCI)
    IBS_avg = mean(IBS)

    return CI_avg, GHCI_avg, IBS_avg, best_CI, best_GHCI, best_IBS, best_fold_rsf

In [8]:
def gridsearch_survival_model(X, t, e):
    """
    Function to run a gridsearch on various Random Survival Forest hyperparameters

    Args:
        X: array -- input features, rows as samples
        t: array -- time labels for X, when event of interest or censoring occurred
        e: array -- event labels for X, if event occurred (1=event, 0=censoring)
    """
    num_tree=(15, 20, 25, 30, 35)
    max_depth=(4, 6, 8, 12, 15)
    min_node=(3, 5, 8, 10)
    num_tree_best = 0
    max_depth_best = 0
    min_node_best = 0
    c_index_best = 0
    ghci_best = 0
    best_rsf = None


    for a in num_tree:
        for b in max_depth:
            for c in min_node:
                CI_avg, GHCI_avg, IBS_avg, best_CI, best_GHCI, best_IBS, best_fold_rsf = \
                    kfold_train_survival_model(X, t, e, num_trees=a, max_depth=b, min_node_size=c, k=5, seed=16)
                # print(a, b, c, CI_avg)
                if CI_avg > c_index_best:
                    c_index_best = CI_avg
                    ghci_best = GHCI_avg
                    num_tree_best = a
                    max_depth_best = b
                    min_node_best = c
                    best_rsf = best_fold_rsf

    return c_index_best, ghci_best, num_tree_best, max_depth_best, min_node_best, best_fold_rsf

# Main Script

### Liver

In [14]:
data_folder = "/Data/FeatureSelection/HCC_MCRC_ICC_HDFS_90_10/"

train_liver_data = pd.read_excel(os.path.join(data_folder, "train_liver_feats_and_labels.xlsx"))
test_liver_data = pd.read_excel(os.path.join(data_folder, "test_liver_feats_and_labels.xlsx"))

features_to_drop=[]
# features_to_drop = ['LeastAxisLength', '90Percentile', 'Contrast', 'MeshVolume', 'Complexity', 'MinorAxisLength', \
#                     '10Percentile', 'Uniformity', 'Mean', 'Energy', 'InterquartileRange', 'TotalEnergy', 'Busyness', \
#                     'RootMeanSquared', 'Maximum3DDiameter']

X_liver = train_liver_data.drop(labels=["ScoutID", "HDFS_Time", "HDFS_Code", "Cancer_Type"], axis=1)
X_liver = X_liver.drop(labels=features_to_drop, axis=1)
t_liver = train_liver_data["HDFS_Time"]
e_liver = train_liver_data["HDFS_Code"]

XT_liver = test_liver_data.drop(labels=["ScoutID", "HDFS_Time", "HDFS_Code", "Cancer_Type"], axis=1)
XT_liver = XT_liver.drop(labels=features_to_drop, axis=1)
tT_liver = test_liver_data["HDFS_Time"]
eT_liver = test_liver_data["HDFS_Code"]


# CI_avg, GHCI_avg, IBS_avg, best_CI, best_GHCI, best_IBS, best_fold_rsf = kfold_train_survival_model(\
#     X_liver, t_liver, e_liver, num_trees=5, max_depth=5, min_node_size=5, k=5, seed=16)


In [15]:
liver_vselected_features = calculate_vif(X_liver, 10)
X_liver = liver_vselected_features

X_liver = cox_feature_select(X_liver, t_liver, e_liver)

/opt/conda/lib/python3.8/site-packages/statsmodels/stats/outliers_influence.py:195: RuntimeWarning: divide by zero encountered in double_scalars
  vif = 1. / (1. - r_squared_i)


VIF remaining variables:
Index(['Kurtosis', 'Skewness', 'Variance', 'Busyness', 'Coarseness',
       'Complexity', 'Contrast', 'Strength', 'Flatness', 'SurfaceArea'],
      dtype='object')
Cox filter remaining variables: 
 Index(['Contrast', 'Flatness', 'Strength', 'Skewness', 'Kurtosis', 'Busyness',
       'Complexity', 'Variance', 'SurfaceArea'],
      dtype='object')


/opt/conda/lib/python3.8/site-packages/lifelines/utils/__init__.py:1103: ConvergenceWarning: Column(s) ['Coarseness', 'Contrast'] have very low variance. This may harm convergence. 1) Are you using formula's? Did you mean to add '-1' to the end. 2) Try dropping this redundant column before fitting if convergence fails.

  warnings.warn(dedent(warning_text), ConvergenceWarning)


In [12]:
liv_c_index_best, liv_ghci_best, liv_num_tree_best, liv_max_depth_best, liv_min_node_best, liv_best_fold_rsf = gridsearch_survival_model(X_liver, t_liver, e_liver)

print("Best c-index:",liv_c_index_best)
# print(ghci_best)
print("Best num_tree val:",liv_num_tree_best)
print("Best max_depth val:", liv_max_depth_best)
print("Best min_node val:",liv_min_node_best)

liver_rsf = train_survival_model(X_liver, t_liver, e_liver, num_trees=liv_num_tree_best, max_depth=liv_max_depth_best, min_node_size=liv_min_node_best, seed=16)

train_liver_cind, train_liver_ghci, train_liver_ibs, train_liver_riskpred = evaluate_forest(liver_rsf, X_liver, t_liver, e_liver)

liver_h_c_ind, liver_gh_c_ind, liver_ibs, liver_riskpreds = evaluate_forest(liver_rsf, XT_liver, tT_liver, eT_liver)

print("Training: ")
print("Harrel's C-index: ", train_liver_cind)
print("GH C-index: ", train_liver_ghci)
print("IBS: ", train_liver_ibs)

print()
print("Testing: ")
print("Harrel's C-index: ", liver_h_c_ind)
print("GH C-index: ", liver_gh_c_ind)
print("IBS: ", liver_ibs)

liv_var_imps = liver_rsf.variable_importance_table
liv_var_imps

Best c-index: 0.5556408057387195
Best num_tree val: 20
Best max_depth val: 12
Best min_node val: 10
Training: 
Harrel's C-index:  0.7014636326863515
GH C-index:  0.9963068333602921
IBS:  0.17139755800635026

Testing: 
Harrel's C-index:  0.6052602858850161
GH C-index:  0.6710340818219738
IBS:  0.24852712227343923


,feature,importance,pct_importance
0,Skewness,5.498713,0.492855
1,SurfaceArea,2.069212,0.185465
2,Strength,1.861589,0.166856
3,Contrast,0.785553,0.070410
4,Flatness,0.748674,0.067104
5,Variance,0.193120,0.017310
6,Complexity,-0.045139,0.000000
7,Busyness,-0.648718,0.000000
8,Kurtosis,-0.937405,0.000000


In [13]:
test_liver_predictions = test_liver_data.loc[:, ['ScoutID', "HDFS_Time", "HDFS_Code", "Cancer_Type"]]
test_liver_predictions['Prediction'] = liver_riskpreds
test_liver_predictions.to_excel(os.path.join(data_folder, "RSF_test_liver_predictions_90_10.xlsx"), index=False)

### Tumor

In [14]:
# Data Loading and Setup
data_folder = "/Data/FeatureSelection/HCC_MCRC_ICC_HDFS_90_10/"

train_tumor_data = pd.read_excel(os.path.join(data_folder, "train_tumor_feats_and_labels.xlsx"))
test_tumor_data = pd.read_excel(os.path.join(data_folder, "test_tumor_feats_and_labels.xlsx"))

features_to_drop = []
# features_to_drop = ['SurfaceVolumeRatio', 'Maximum2DDiameterSlice', 'MajorAxisLength', 'Busyness', 'VoxelVolume',\
                    # '90Percentile', 'Median', 'Energy', 'RootMeanSquared']

X_tumor = train_tumor_data.drop(labels=["ScoutID", "HDFS_Time", "HDFS_Code", "Cancer_Type"], axis=1)
X_tumor = X_tumor.drop(labels=features_to_drop, axis=1)
t_tumor = train_tumor_data["HDFS_Time"]
e_tumor = train_tumor_data["HDFS_Code"]

XT_tumor = test_tumor_data.drop(labels=["ScoutID", "HDFS_Time", "HDFS_Code", "Cancer_Type"], axis=1)
XT_tumor = XT_tumor.drop(labels=features_to_drop, axis=1)
tT_tumor = test_tumor_data["HDFS_Time"]
eT_tumor = test_tumor_data["HDFS_Code"]

In [15]:
# Feature selection
tum_vselected_features = calculate_vif(X_tumor, 10)
X_tumor = tum_vselected_features

X_tumor = cox_feature_select(X_tumor, t_tumor, e_tumor)

/opt/conda/lib/python3.8/site-packages/statsmodels/stats/outliers_influence.py:195: RuntimeWarning: divide by zero encountered in double_scalars
  vif = 1. / (1. - r_squared_i)


VIF remaining variables:
Index(['10Percentile', 'Kurtosis', 'Minimum', 'TotalEnergy', 'Busyness',
       'Coarseness', 'Complexity', 'Contrast', 'Strength', 'Flatness',
       'Maximum2DDiameterColumn', 'MeshVolume'],
      dtype='object')
Cox filter remaining variables: 
 Index(['Contrast', 'Flatness', 'Strength', 'Maximum2DDiameterColumn',
       '10Percentile', 'Kurtosis', 'Busyness', 'Complexity', 'Minimum',
       'MeshVolume', 'TotalEnergy'],
      dtype='object')


/opt/conda/lib/python3.8/site-packages/lifelines/utils/__init__.py:1103: ConvergenceWarning: Column(s) ['Coarseness'] have very low variance. This may harm convergence. 1) Are you using formula's? Did you mean to add '-1' to the end. 2) Try dropping this redundant column before fitting if convergence fails.

  warnings.warn(dedent(warning_text), ConvergenceWarning)


In [16]:
# RSF Model creation and evaluation
tum_c_index_best, tum_ghci_best, tum_num_tree_best, tum_max_depth_best, tum_min_node_best, tum_best_fold_rsf = gridsearch_survival_model(X_tumor, t_tumor, e_tumor)

print("Best k-fold c-index:",tum_c_index_best)
# print(ghci_best)
print("Best num_tree val:",tum_num_tree_best)
print("Best max_depth val:", tum_max_depth_best)
print("Best min_node val:",tum_min_node_best)

tumor_rsf = train_survival_model(X_tumor, t_tumor, e_tumor, num_trees=tum_num_tree_best, max_depth=tum_max_depth_best, min_node_size=tum_min_node_best, seed=16)
train_tumor_cind, train_tumor_ghci, train_tumor_ibs, train_tumor_riskpred = evaluate_forest(tumor_rsf, X_tumor, t_tumor, e_tumor)
tumor_h_c_ind, tumor_gh_c_ind, tumor_ibs, tumor_preds = evaluate_forest(tumor_rsf, XT_tumor, tT_tumor, eT_tumor)

print("Training: ")
print("Harrel's C-index: ", train_tumor_cind)
print("GH C-index: ", train_tumor_ghci)
print("IBS: ", train_tumor_ibs)

print()
print("Testing: ")
print("Harrel's C-index: ", tumor_h_c_ind)
print("GH C-index: ", tumor_gh_c_ind)
print("IBS: ", tumor_ibs)

tum_var_imps = tumor_rsf.variable_importance_table
tum_var_imps

Best k-fold c-index: 0.5985046356357657
Best num_tree val: 20
Best max_depth val: 4
Best min_node val: 3
Training: 
Harrel's C-index:  0.6313905604015215
GH C-index:  0.991600139487711
IBS:  0.19522905346033947
Harrel's C-index:  0.6328024720032681
GH C-index:  0.6940854107784103
IBS:  0.282533349505983


,feature,importance,pct_importance
0,Maximum2DDiameterColumn,4.380915,0.326096
1,Kurtosis,2.880566,0.214416
2,MeshVolume,2.009008,0.149542
3,Flatness,1.470401,0.109450
4,Minimum,1.334858,0.099361
5,TotalEnergy,0.958495,0.071346
6,Complexity,0.176317,0.013124
7,10Percentile,0.127451,0.009487
8,Strength,0.096433,0.007178
9,Busyness,-0.078841,0.000000


In [17]:
test_tumor_predictions = test_tumor_data.loc[:, ['ScoutID', "HDFS_Time", "HDFS_Code", "Cancer_Type"]]
test_tumor_predictions['Prediction'] = tumor_preds
test_tumor_predictions.to_excel(os.path.join(data_folder, "RSF_test_tumor_predictions_90_10.xlsx"), index=False)

### Liver and Tumor

In [13]:
data_folder = "/Data/FeatureSelection/HCC_MCRC_ICC_HDFS_90_10/"

train_livertumor_data = pd.read_excel(os.path.join(data_folder, "train_liver_tumor_feats_and_labels.xlsx"))
test_livertumor_data = pd.read_excel(os.path.join(data_folder, "test_liver_tumor_feats_and_labels.xlsx"))

features_to_drop = []
# features_to_drop = ['Strength', 'MinorAxisLength', 'VoxelVolume', 'Complexity', 'Uniformity', 'Minimum', \
#                     'Median', 'InterquartileRange', 'Energy']

X_livertumor = train_livertumor_data.drop(labels=["ScoutID", "HDFS_Time", "HDFS_Code", "Cancer_Type"], axis=1)
X_livertumor = X_livertumor.drop(labels=features_to_drop, axis=1)
t_livertumor = train_livertumor_data["HDFS_Time"]
e_livertumor = train_livertumor_data["HDFS_Code"]

XT_livertumor = test_livertumor_data.drop(labels=["ScoutID", "HDFS_Time", "HDFS_Code", "Cancer_Type"], axis=1)
XT_livertumor = XT_livertumor.drop(labels=features_to_drop, axis=1)
tT_livertumor = test_livertumor_data["HDFS_Time"]
eT_livertumor = test_livertumor_data["HDFS_Code"]

In [14]:
livtum_vselected_features = calculate_vif(X_livertumor, 10)
X_livertumor = livtum_vselected_features

X_livertumor = cox_feature_select(X_livertumor, t_livertumor, e_livertumor)

/opt/conda/lib/python3.8/site-packages/statsmodels/stats/outliers_influence.py:195: RuntimeWarning: divide by zero encountered in double_scalars
  vif = 1. / (1. - r_squared_i)


VIF remaining variables:
Index(['Kurtosis', 'Skewness', 'TotalEnergy', 'Variance', 'Busyness',
       'Complexity', 'Contrast', 'Strength', 'Flatness', 'SurfaceVolumeRatio'],
      dtype='object')
Cox filter remaining variables: 
 Index(['SurfaceVolumeRatio', 'Flatness', 'Skewness', 'Strength', 'Kurtosis',
       'Busyness', 'Complexity', 'Variance', 'TotalEnergy'],
      dtype='object')


/opt/conda/lib/python3.8/site-packages/lifelines/utils/__init__.py:1103: ConvergenceWarning: Column(s) ['Contrast'] have very low variance. This may harm convergence. 1) Are you using formula's? Did you mean to add '-1' to the end. 2) Try dropping this redundant column before fitting if convergence fails.

  warnings.warn(dedent(warning_text), ConvergenceWarning)


In [15]:
livtum_c_index_best, livtum_ghci_best, livtum_num_tree_best, livtum_max_depth_best, livtum_min_node_best, livtum_best_fold_rsf = \
    gridsearch_survival_model(X_livertumor, t_livertumor, e_livertumor)

print("Best k-fold c-index:",livtum_c_index_best)
# print(ghci_best)
print("Best num_tree val:",livtum_num_tree_best)
print("Best max_depth val:",livtum_max_depth_best)
print("Best min_node val:",livtum_min_node_best)

livertumor_rsf = train_survival_model(X_livertumor, t_livertumor, e_livertumor, num_trees=livtum_num_tree_best, max_depth=livtum_max_depth_best, min_node_size=livtum_min_node_best, seed=16)

train_livertumor_cind, train_livertumor_ghci, train_livertumor_ibs, train_livertumor_riskpred = evaluate_forest(livertumor_rsf, X_livertumor, t_livertumor, e_livertumor)

livertumor_h_c_ind, livertumor_gh_c_ind, livertumor_ibs, livertumor_riskpreds = evaluate_forest(livertumor_rsf, XT_livertumor, tT_livertumor, eT_livertumor)

print("Training: ")
print("Harrel's C-index: ", train_livertumor_cind)
print("GH C-index: ", train_livertumor_ghci)
print("IBS: ", train_livertumor_ibs)

print()
print("Testing: ")
print("Harrel's C-index: ",livertumor_h_c_ind)
print("GH C-index: ",livertumor_gh_c_ind)
print("IBS: ",livertumor_ibs)

var_imps = livertumor_rsf.variable_importance_table
var_imps

Best k-fold c-index: 0.5956934851792524
Best num_tree val: 25
Best max_depth val: 4
Best min_node val: 5
Training: 
Harrel's C-index:  0.6502548268094505
GH C-index:  0.9947531226284709
IBS:  0.19649582923465644

Testing: 
Harrel's C-index:  0.6024511048260658
GH C-index:  0.8702756685083678
IBS:  0.24770199890714759


,feature,importance,pct_importance
0,Skewness,3.941655,0.302668
1,Variance,2.709866,0.208082
2,Complexity,1.856625,0.142565
3,Strength,1.800248,0.138236
4,Kurtosis,1.165461,0.089492
5,SurfaceVolumeRatio,0.765052,0.058746
6,Busyness,0.539746,0.041445
7,Flatness,0.244388,0.018766
8,TotalEnergy,-0.203890,0.000000


In [12]:
test_livertumor_predictions = test_livertumor_data.loc[:, ['ScoutID', "HDFS_Time", "HDFS_Code", "Cancer_Type"]]
test_livertumor_predictions['Prediction'] = livertumor_riskpreds
test_livertumor_predictions.to_excel(os.path.join(data_folder, "RSF_test_liver_tumor_predictions_90_10.xlsx"), index=False)

# Other Code

In [ ]:
# correlation_matrix(X_liver, figure_size=(30,15), text_fontsize=10)